<a href="https://colab.research.google.com/github/PritishRane/LLM_learning-/blob/main/imdb_llm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

In [2]:
pip install datasets

In [3]:
from transformers import pipeline
print('hugging face is ready')

hugging face is ready


In [4]:
from transformers import pipeline
generator=pipeline('text-generation',model='gpt2')
output=generator('Once upon a time', max_length= 50 , num_return_sequences=1)
print(output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, I realized I was in love with her; I just went in there to tell her how I felt and she laughed it off. "You don't need to lie to get your girl," I said. "You simply have


In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer=AutoTokenizer.from_pretrained('gpt2')
text = "Hello, Hugging Face!"
tokens = tokenizer(text, return_tensors="pt")
print(tokens)

{'input_ids': tensor([[15496,    11, 12905,  2667, 15399,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [7]:
decoded_text = tokenizer.decode(tokens["input_ids"][0])
print(decoded_text)

Hello, Hugging Face!


In [8]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [9]:
ner = pipeline("ner", grouped_entities=True)
result = ner("Hugging Face is based in New York.")
print(result)


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity_group': 'ORG', 'score': 0.88766813, 'word': 'Hugging Face', 'start': 0, 'end': 12}, {'entity_group': 'LOC', 'score': 0.9985268, 'word': 'New York', 'start': 25, 'end': 33}]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [10]:
qa=pipeline("question-answering")
context="Hugging Face is a company creating tools for NLP and ML."
question="Where is Hugging Face based?"
result=qa(context=context,question=question)
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


{'score': 0.42966902256011963, 'start': 45, 'end': 48, 'answer': 'NLP'}


#Project


In [11]:
from datasets import load_dataset
dataset = load_dataset("imdb", split="train[:1000]")
print(dataset[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [12]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Assign a padding token
tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token as the padding token

# Preprocessing function
def preprocess_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding=True, max_length=512)
    tokenized["labels"] = examples["label"]  # Add labels to the tokenized dataset
    return tokenized




In [13]:
from transformers import default_data_collator

data_collator = default_data_collator

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [16]:
from transformers import GPT2ForSequenceClassification
model=GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
pip install evaluate

In [23]:
from transformers import Trainer, TrainingArguments
import evaluate


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [24]:
training_args = TrainingArguments(
    output_dir="./results",         # Directory for model checkpoints
    evaluation_strategy="epoch",    # Evaluate after each epoch
    learning_rate=2e-5,             # Learning rate
    per_device_train_batch_size=1,  # Batch size for training
    per_device_eval_batch_size=1,   # Batch size for evaluation
    num_train_epochs=3,             # Number of training epochs
    weight_decay=0.01,              # Weight decay
)



# Initialize Trainer
trainer = Trainer(
    model=model,                   # Model to train
    args=training_args,            # Training arguments
    train_dataset=tokenized_dataset,  # Training dataset
    eval_dataset=tokenized_dataset,   # Evaluation dataset
    compute_metrics=compute_metrics,  # Metrics for evaluation
    data_collator=data_collator     # Handle padding in batches
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.000000,0.000000,1.000000
2,0.000000,0.000000,1.000000
3,0.000000,0.000000,1.000000


TrainOutput(global_step=3000, training_loss=8.742012103842475e-10, metrics={'train_runtime': 679.9613, 'train_samples_per_second': 4.412, 'train_steps_per_second': 4.412, 'total_flos': 783890251776000.0, 'train_loss': 8.742012103842475e-10, 'epoch': 3.0})

In [27]:
# Load the test dataset
test_dataset = load_dataset("imdb", split="test[:1000]")

# Tokenize the test dataset
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
# Evaluate the model
evaluation_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)

# Print evaluation metrics
print(evaluation_results)


{'eval_loss': 3.576278440053926e-10, 'eval_accuracy': 1.0, 'eval_runtime': 34.2244, 'eval_samples_per_second': 29.219, 'eval_steps_per_second': 29.219, 'epoch': 3.0}


In [38]:
text = "This movie was absolutely fantastic!"

# Tokenize the input
inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)

# Perform prediction
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class = logits.argmax(dim=-1).item()

# Print predicted class
print(f"Predicted class: {predicted_class}")

Predicted class: 0


In [39]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json',
 './fine_tuned_gpt2/tokenizer.json')

In [40]:
from transformers import GPT2ForSequenceClassification, AutoTokenizer

# Load the fine-tuned model and tokenizer
model = GPT2ForSequenceClassification.from_pretrained("./fine_tuned_gpt2")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt2")
